<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install spark

In [2]:
!pip install pyspark
!pip install findspark

In [3]:
# Start session
import findspark
findspark.init()


In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
sc = SparkContext()
spark = SparkSession \
    .builder \
    .appName("e-commerce web server") \
    .getOrCreate()

25/04/17 02:36:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/17 02:36:05 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [16]:

!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2025-04-17 02:36:22--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv.2’

searchterms.csv.2   100%[===================>] 227.99K  --.-KB/s    in 0.006s  

2025-04-17 02:36:23 (39.6 MB/s) - ‘searchterms.csv.2’ saved [233457/233457]



In [7]:
# Load the csv into a spark dataframe

In [21]:
df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)

In [8]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [30]:
print(f"Number of rows {len(df.columns)}\nNumber of columns {df.count()}") 

Number of rows 4
Number of columns 10000


In [9]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [32]:
df.head(5)

[Row(day=12, month=11, year=2021, searchterm='mobile 6 inch'),
 Row(day=12, month=11, year=2021, searchterm='mobile latest'),
 Row(day=12, month=11, year=2021, searchterm='tablet wifi'),
 Row(day=12, month=11, year=2021, searchterm='laptop 14 inch'),
 Row(day=12, month=11, year=2021, searchterm='mobile 5g')]

In [10]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [33]:
df.printSchema()

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [11]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [37]:
df.createOrReplaceTempView("ecommerce")
result = spark.sql("SELECT COUNT(*) as GL_search FROM ecommerce WHERE searchterm='gaming laptop'")
result.show()

+---------+
|GL_search|
+---------+
|      499|
+---------+



In [12]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [42]:
result2 = spark.sql("SELECT searchterm,COUNT(searchterm) as frequency FROM ecommerce GROUP BY searchterm ORDER BY COUNT(searchterm) DESC LIMIT 5").show()

[Stage 34:============================================>         (166 + 8) / 200]

+-------------+---------+
|   searchterm|frequency|
+-------------+---------+
|mobile 6 inch|     2312|
|    mobile 5g|     2301|
|mobile latest|     1327|
|       laptop|      935|
|  tablet wifi|      896|
+-------------+---------+



In [44]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!wget  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!tar –xvzf model.tar.gz

--2025-04-17 02:53:17--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz.1’

model.tar.gz.1      100%[===================>]   1.46K  --.-KB/s    in 0s      

2025-04-17 02:53:17 (15.2 MB/s) - ‘model.tar.gz.1’ saved [1490/1490]

tar: invalid option -- '�'
Try 'tar --help' or 'tar --usage' for more information.


In [14]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [48]:
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load("sales_prediction.model")

In [15]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [67]:
print(model.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2) (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. (default: 1.35)
featuresCol: features column name (current: features)
fitIntercept: whether to fit an intercept term (default: True)
labelCol: label column name (current: sales)
loss: The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError) (default: squaredError)
maxIter: maximum number of iterations (>= 0) (current: 100)
predictionCol: prediction column name (default: prediction)
regParam: regularization parameter (>= 0) (current: 0.1)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (Default auto) (default: auto)
standardization: whether to standardize the training features before fitting the mo

In [93]:
from pyspark.ml.feature import VectorAssembler
def predict(year):
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")  # Adjusted input column name
    data = [[year]]  # Changed input to reflect height
    columns = ["year"]  # Updated column names for clarity
    df = spark.createDataFrame(data, columns)
    transformed_df = assembler.transform(df)  # Updated column selection
    transformed_df.show()
    predictions = model.transform(transformed_df)
    predictions.select('*').show()

In [94]:
predict(2023)

+----+--------+
|year|features|
+----+--------+
|2023|[2023.0]|
+----+--------+

+----+--------+------------------+
|year|features|        prediction|
+----+--------+------------------+
|2023|[2023.0]|175.16564294006457|
+----+--------+------------------+

